In [53]:
## 시작하기 전에
## 외부데이터를 통한 DBSCAN clustering을 실습해보자  
## https://yssa.tistory.com/54 참고 
import pandas as pd 
import folium 

# 학교 알리미 공개용 데이터 
# 서울시 중학생 졸업생 진로현황 데이터셋 
# 고등학교 진학률 데이터를 활용하여 속성이 비슷한 중학교끼리 클러스터를 만들어 보는 실습
file_path = 'data/2016_middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path, header=0)
print(df.columns.values)
print(df.info())
print(df.head())
print(df.describe())

['Unnamed: 0' '지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고'
 '외고_국제고' '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
Unnamed: 0    415 non-null int64
지역            415 non-null object
학교명           415 non-null object
코드            415 non-null int64
유형            415 non-null object
주야            415 non-null object
남학생수          415 non-null int64
여학생수          415 non-null int64
일반고           415 non-null float64
특성화고          415 non-null float64
과학고           415 non-null float64
외고_국제고        415 non-null float64
예고_체고         415 non-null float64
마이스터고         415 non-null float64
자사고           415 non-null float64
자공고           415 non-null float64
기타진학          415 non-null float64
취업            415 non-null int64
미상            415 non-null float64
위도            415 non-null float64
경도            415 non-null float64
dtypes: float64(12), int64(5), obje

In [54]:
# 지도에 위치 표시
mschool_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', zoom_start=12)

# 중학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                        radius=5,              # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7,      # 투명도    
                        popup=name
    ).add_to(mschool_map)
#지도를 html파일로 저장하기 
mschool_map.save('data/seoul_mschool_location.html')


In [56]:
# 전처리 실시 

# sklearn 라이브러리에서 cluster 군집 모형 가져오기 
from sklearn import cluster 
# cluster = DBSCAN(n_jobs=-1) # DBSCAN 객체 생성

# 분석에 사용할 속성 선택(과학고, 외고국제고, 자사고 진학률)
columns_list = [10,11,14]
X = df.iloc[:, columns_list]
print(X[:5])

# 설명 변수 데이터를 정규화 
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit(X).transform(X)

dbm = cluster.DBSCAN(eps=0.2, min_samples=5) #반지름과 원에 포함될 최소 데이터 개수 최소값 설정 
dbm.fit(X)

cluster_label = dbm.labels_
print(cluster_label) #outlier를 제외하고 3개로 클러스터링됨 


     과학고  외고_국제고    자사고
0  0.018   0.007  0.227
1  0.000   0.035  0.043
2  0.009   0.012  0.090
3  0.013   0.013  0.065
4  0.007   0.010  0.282
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  2 -1  0 -1
 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  0  3 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  1  0
 -1 -1 -1  0 -1 -1 -1 -1  0 -1  0  0 -1 -1  0 -1 -1 -1  0  0 -1 -1  0 -1
 -1 -1  0 -1 -1 -1  0  2  0  0  0  0  0 -1 -1 -1  0 -1  0 -1 -1  0 -1  0
 -1  0  0 -1 -1 -1 -1  1  0 -1  0  0 -1 -1 -1  0 -1 -1 -1 -1 -1  0  1 -1
 -1  0  2  0 -1 -1  1 -1 -1 -1  0  0  0 -1 -1  0 -1 -1 -1  0  0 -1 -1 -1
 -1  0 -1 -1 -1  0 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1  0 -1  0  0 -1 -1
 -1 -1 -1  0 -1 -1 -1  1  0  3  1 -1  0  0 -1  0 -1 -1  0  0  2 -1 -1  3
  0  0 -1 -1 -1 -1  0 -1  0  0 -1  0  0  0 -1 -1  0 -1 -1 -1 -1 -1  2  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0  0 -1 -1  0 -1  3  0  2 -1 -1
 -1 -1  0 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1  1 -1  

In [57]:
# 예측 결과를 데이터프레임에 추가 
df['Cluster'] = cluster_label
print(df.head())

grouped_cols = [0,1,3] + columns_list
grouped = df.groupby('Cluster')
for key, group in grouped:
    print('* key:', key)
    print('* number:', len(group))
    print(group.iloc[:, grouped_cols].head())
    print('\n')

'''
클러스터 0은 외고(국제고)와 자사고 합격률은 높지만 과학고 합격자가 없다 
클러스터1은 자사고 합격자만 존재하는 그룹이고,
클러스터2는 자사고 합격률이 매우 높으면서 과학고와 외고(국제고) 합격자도 일부 존재한다. 
클러스터3은 과학고 합격자가 없이 외고(국제고)와 자사고 합격자를 배출한 점 
'''

   Unnamed: 0   지역               학교명  코드  유형  주야  남학생수  여학생수    일반고   특성화고  \
0           0  성북구    서울대학교사범대학부설중학교   3  국립  주간   277     0  0.585  0.148   
1           1  종로구  서울대학교사범대학부설여자중학교   3  국립  주간     0   256  0.680  0.199   
2           2  강남구             개원중학교   3  공립  주간   170   152  0.817  0.047   
3           3  강남구             개포중학교   3  공립  주간    83    72  0.755  0.097   
4           4  서초구             경원중학교   3  공립  주간   199   212  0.669  0.017   

   ...  예고_체고  마이스터고    자사고    자공고   기타진학  취업     미상         위도          경도  \
0  ...  0.000  0.011  0.227  0.000  0.004   0  0.000  37.594942  127.038909   
1  ...  0.008  0.000  0.043  0.004  0.031   0  0.000  37.577473  127.003857   
2  ...  0.003  0.006  0.090  0.003  0.009   0  0.003  37.491637  127.071744   
3  ...  0.019  0.019  0.065  0.000  0.019   0  0.000  37.480439  127.062201   
4  ...  0.005  0.000  0.282  0.000  0.010   0  0.000  37.510750  127.008900   

   Cluster  
0       -1  
1       -1  
2       -1  
3   

'\n클러스터 0은 외고(국제고)와 자사고 합격률은 높지만 과학고 합격자가 없다 \n클러스터1은 자사고 합격자만 존재하는 그룹이고,\n클러스터2는 자사고 합격률이 매우 높으면서 과학고와 외고(국제고) 합격자도 일부 존재한다. \n클러스터3은 과학고 합격자가 없이 외고(국제고)와 자사고 합격자를 배출한 점 \n'

In [58]:
# 데이터 시각화로 좀더 보기좋게 구분하자
colors = {-1:'gray', 0:'orange', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'coral', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan'}

cluster_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                         zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster_map)

cluster_map.save('data/seoul_mschool_cluster.html') #파일링크로 저장 


In [59]:
# type(국립, 공립, 사립)열을 추가해서 다시 클러스터링 실시 
# df.iloc[:, [8,9,10,11,12,13,14,15,16,17,18]]

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
onehot_type=label_encoder.fit_transform(df['유형'])

# df['type']=df.iloc[:,4]
df['type']=onehot_type
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 23 columns):
Unnamed: 0    415 non-null int64
지역            415 non-null object
학교명           415 non-null object
코드            415 non-null int64
유형            415 non-null object
주야            415 non-null object
남학생수          415 non-null int64
여학생수          415 non-null int64
일반고           415 non-null float64
특성화고          415 non-null float64
과학고           415 non-null float64
외고_국제고        415 non-null float64
예고_체고         415 non-null float64
마이스터고         415 non-null float64
자사고           415 non-null float64
자공고           415 non-null float64
기타진학          415 non-null float64
취업            415 non-null int64
미상            415 non-null float64
위도            415 non-null float64
경도            415 non-null float64
Cluster       415 non-null int64
type          415 non-null int32
dtypes: float64(12), int32(1), int64(6), object(4)
memory usage: 73.1+ KB
None


In [60]:
# 분석에 사용할 속성을 선택 (과학고, 외고_국제고, 자사고 진학률)
columns_list2 = [10,11,14,22]
X2 = df.iloc[:, columns_list2]
print(X[:5])
# 설명 변수 데이터를 정규화
X2 = StandardScaler().fit(X2).transform(X2)

dbm2 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm2.fit(X2)   
 
cluster_label2 = dbm2.labels_   
print(cluster_label2)

# 예측 결과를 데이터프레임에 추가
df['Cluster2'] = cluster_label2
print(df.head())

[[ 2.02375287 -0.57972902  1.84751715]
 [-0.65047921  1.84782097 -0.48039958]
 [ 0.68663683 -0.14623795  0.11423133]
 [ 1.28091062 -0.05953974 -0.20206171]
 [ 0.38949993 -0.31963438  2.54336183]]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1  0 -1
 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  7 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  2  1
 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  9  1 -1 -1 -1 -1
 -1 -1  1 -1 -1 -1  1 -1  1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  1
 -1  1  1 -1 -1 -1 -1  3  1 -1  1  1 -1 -1 -1  1 -1 -1 -1 -1 -1  1  3 -1
 -1  1  4  1 -1 -1  6 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1  1 -1 -1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1 -1 -1  9 -1  0  1 -1 -1
 -1 -1 -1  1 -1 -1 -1  5 -1  7  6 -1  1 -1 -1  1 -1 -1  1  1  4 -1 -1  7
  1  1 -1 -1 -1 -1  8 -1 -1  1 -1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  4  1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  9 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1  1 -1 -1 -1 -1  7  1  4 -1 -1
 

In [62]:
# 이하 시각화 동일 
grouped_cols2 = [0, 1, 3] + columns_list2
grouped2 = df.groupby('Cluster2')
for key, group in grouped2:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped_cols2].head())
    print('\n')

#클러스터 0은 외고(국제고)와 자사고 합격률은 높지만 과학고 합격자가 없다
#클러스터 1은 자사고 합격자만 존재하는 그룹이고,
#클러스터 2는 자사고 합격률이 매우 높으면서 과학고와 외고(국제고) 합격자도 일부 존재한다.
#클러스터 3은 과학고 합격자 없이 외고(국제고)와 자사고 합격자를 배출한 점

colors = {-1:'gray', 0:'orange', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'coral', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan'}

cluster_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster2):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster_map)
cluster_map.save('data/seoul_mschool_cluster2.html')

* key : -1
* number : 281
   Unnamed: 0   지역  코드    과학고  외고_국제고    자사고  type
0           0  성북구   3  0.018   0.007  0.227     1
1           1  종로구   3  0.000   0.035  0.043     1
2           2  강남구   3  0.009   0.012  0.090     0
3           3  강남구   3  0.013   0.013  0.065     0
4           4  서초구   3  0.007   0.010  0.282     0


* key : 0
* number : 8
     Unnamed: 0   지역  코드  과학고  외고_국제고    자사고  type
22           22  강남구   3  0.0   0.019  0.044     0
43           43  송파구   3  0.0   0.021  0.054     0
51           51  송파구   3  0.0   0.021  0.068     0
93           93  강서구   3  0.0   0.019  0.057     0
164         164  중랑구   3  0.0   0.020  0.062     0


* key : 1
* number : 59
    Unnamed: 0   지역  코드  과학고  외고_국제고    자사고  type
28          28  서초구   3  0.0   0.015  0.050     0
47          47  강동구   3  0.0   0.010  0.026     0
58          58  강동구   3  0.0   0.013  0.026     0
62          62  강동구   3  0.0   0.009  0.031     0
67          67  송파구   3  0.0   0.015  0.072     0


* key : 2